In [ ]:
import importlib
import functions.core_functions as core_functions
import dask
import dask.dataframe as dd
from dask.dataframe.utils import assert_eq
import pandas as pd
import numpy as np
import pandas_gbq
import json
import gc
import cudf
import os
from datetime import datetime, timedelta

dask.config.set({"dataframe.backend": "cudf"})

importlib.reload(core_functions)

In [ ]:
init = {}
init = core_functions.initialize_clients(service_account_secret_name='SA_N90_CORE_APPS')
init_2 = core_functions.initialize_clients(service_account_secret_name='SA_ADHOC_BILLING')

config = init.get('config')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config.get('SA_N90_CORE_APPS')
bigquery_client = init.get('clients').get('bigquery_client')
storage_client = init.get('clients').get('storage_client')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config.get('SA_ADHOC_BILLING')
bigquery_client_2 = init_2.get('clients').get('bigquery_client')

In [ ]:
init.get('clients')


In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config.get('SA_ADHOC_BILLING')
broadcast_cal_sql = f"""
    SELECT id as bcw_id, bcw_index, bcm_index, bcw_start_date, bcw_end_date FROM `adhoc-billing.avs_billing_process.lu_broadcast_week`
"""
broadcast_cal_df = core_functions.fetch_gbq_data(query=broadcast_cal_sql, bigquery_client=bigquery_client_2)

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config.get('SA_N90_CORE_APPS')
geo_sql = f"""
    SELECT * from `next90-core-applications.next90_analytics.geos` WHERE geo_type = 'dma'
"""
geo_df = core_functions.fix_df_dtypes(core_functions.fetch_gbq_data(query=geo_sql, bigquery_client=bigquery_client))

int_cols = ['geo_location', 'geo_neustar_id','geo_us_msa_id', 'geo_us_county_fips_id','geo_ca_cma_id']
for col in int_cols:
    geo_df[col] = geo_df[col].fillna(-1).astype('Int64')

In [ ]:
min_date = '2024-08-01'
max_date = '2025-01-01'
process_month_groups = []
process_month_group = '2024-09'

In [ ]:
session_sql = f"""
select *, EXTRACT(YEAR FROM created_time) AS year, EXTRACT(MONTH FROM created_time) as month, EXTRACT(DAY FROM created_time) as day,
        CONCAT(EXTRACT(YEAR FROM created_time),'-',FORMAT('%02d', EXTRACT(MONTH FROM created_time))) as process_month_group,
        -- Calculate week_within_month
        CAST((EXTRACT(DAY FROM created_time) - 1) / 7 + 1 AS INT64) AS week_within_month 
from `next90-core-applications.n90_data_lake.activity_sessions`
WHERE created_time >= '{min_date}' AND created_time < '{max_date}'
        AND CONCAT(EXTRACT(YEAR FROM created_time),'-',FORMAT('%02d', EXTRACT(MONTH FROM created_time))) = '{process_month_group}'
"""

session_df = core_functions.fetch_gbq_data(session_sql,bigquery_client)

In [ ]:
# del session_df
# gc.collect()

In [ ]:
session_df

In [ ]:
for col in session_df.columns:
    print(f'col {col} has dtype {session_df[col].dtype}, sample value {session_df[col].iloc[0]}')

In [ ]:
session_df['lat'] = session_df['lat'].astype('Float64')
session_df['lon'] = session_df['lat'].astype('Float64')
session_df
core_functions.print_dataframe_python_schema(session_df, 'activity_sessions')